## Import Modules

In [1]:
import dataset as ds
import config as cfg

import torch
from ultralytics import settings
from ultralytics import YOLO
from clearml import Task
import clearml
clearml.browser_login()
import gc


# ClearML
https://app.clear.ml/dashboard

# Yolo Config

In [5]:
# Ultralytics settings
settings.reset()
settings.update({'datasets_dir': cfg.YOLO_DATASET_DIR.replace('/','\\')})
print(settings)

# GPU Utilisation
print(torch.cuda.is_available())
device = torch.device("cuda")
print(torch.backends.cudnn.version())

#Disable Data Augmentation (potentially)
#hsv_h: 0.0  # hue
#hsv_s: 0.0  # saturation
#hsv_v: 0.0  # value
#degrees: 0.0  # rotation
#translate: 0.0  # translate
#scale: 0.0  # scale
#shear: 0.0  # shear
#perspective: 0.0  # perspective
#flipud: 0.0  # flip up-down
#fliplr: 0.0  # flip left-right
#mosaic: 0.0  # mosaic
#mixup: 0.0  # mixup


{'settings_version': '0.0.4', 'datasets_dir': 'C:\\github\\Third-Year-Project\\Intruder-Aircraft-Detection\\datasets\\YOLOv8', 'weights_dir': 'c:\\github\\Third-Year-Project\\Intruder-Aircraft-Detection\\weights', 'runs_dir': 'c:\\github\\Third-Year-Project\\Intruder-Aircraft-Detection\\runs', 'uuid': 'fdb5c10788ffaa41a9047dc764dd8a0a3287d6bec12c0c66234ac745c0366efa', 'sync': True, 'api_key': '', 'openai_api_key': '', 'clearml': True, 'comet': True, 'dvc': True, 'hub': True, 'mlflow': True, 'neptune': True, 'raytune': True, 'tensorboard': True, 'wandb': True}
True
8801


## Load Dataset

In [6]:
# Base paths for the images and labels
train_images_path = f'{cfg.BASE_DATASET}/images/train'
train_labels_path = f'{cfg.BASE_DATASET}/labels/train'
val_images_path = f'{cfg.BASE_DATASET}/images/valid'
val_labels_path = f'{cfg.BASE_DATASET}/labels/valid'

# Base path for metadata
metadata_path = f'{cfg.BASE_DATASET}/metadata.json'

# Create the DataFrames for the train and validation sets
train_df = ds.create_dataframe(train_images_path, train_labels_path, metadata_path)
valid_df = ds.create_dataframe(val_images_path, val_labels_path, metadata_path)

#train_df.info()

In [6]:
# augmentation metadata

#augmentation_metadata = {
#    'methods': {
#        'hsv': {
#            'parameters': {
#                'type': 'h',  # Probability of applying the augmentation
#                'shift': 50,
#                'p': 1.0
#            },
#            'apply_to_percentage': 0.1  # 50% of the training images
#        },
#        'flip': {
#            'parameters': {
#                'orientation': 'h',  # Could be 'h' for horizontal or 'v' for vertical
#                'p': 1.0  # Probability of applying the augmentation
#            },
#            'apply_to_percentage': 0.1  # 50% of the training images
#        },
#        'rotate': {
#            'parameters': {
#                'angle': 90,  # Rotation angle
#                'p': 1.0  # Probability of applying the augmentation
#            },
#            'apply_to_percentage': 0.1  # 30% of the training images
#        },
#        'bnc': {
#            'parameters': {
#                'alpha': 1.5,  # contrast control
#                'beta': 50, # brightness control
#            },
#            'apply_to_percentage': 0.1  # 50% of the training images
#        },
#        'histEq': {
#            'parameters': {
#                'p': 1.0  # Probability of applying the augmentation
#            },
#            'apply_to_percentage': 0.1  # 50% of the training images
#        },
#        'whiteBal': {
#            'parameters': {
#            },
#            'apply_to_percentage': 0.1  # 50% of the training images
#        },
#        'sharpen': {
#            'parameters': {
#            },
#            'apply_to_percentage': 0.1  # 50% of the training images
#        },
#        'gaussian': {
#            'parameters': {
#                'var_limit': (10.0, 50.0) # range of noise varaince
#            },
#            'apply_to_percentage': 0.1  # 50% of the training images
#        },
#        'zoom': {
#            'parameters': {
#                'zoom_factor': 3  # Probability of applying the augmentation
#            },
#            'apply_to_percentage': 0.1  # 50% of the training images
#        }       
#    }
#}


# Test Parameters

In [7]:
# name dataset to be create as well as task name
dataset_name = 'test500'

# project and task constant parameters
project_name= cfg.PROJECT_NAME + '/YOLOv8'
dataset_dir = f'{cfg.YOLO_DATASET_DIR}/{dataset_name}'
project_dir = f'{cfg.YOLO_PROJECT_DIR}/{dataset_name}/'
class_names = cfg.YOLO_CLASS_NAMES

# task specifc training parameters
epochs = 20
model_variant = "yolov8n"

"""
Methods: 
 - flip
 - rotate
 - bnc
 - gaussian
 - histEq
 - whiteBal
 - sharpen
 - zoom
 - hsv
"""

augmentation_metadata = {
    'methods': {        
        'flip': {
            'parameters': {
                'orientation': 'h',  # Could be 'h' for horizontal or 'v' for vertical
                'p': 1.0  # Probability of applying the augmentation
            },
            'apply_to_percentage': 0.5  # 50% of the training images
        }        
    }
}

# Create Datasets

In [8]:
from sklearn.model_selection import train_test_split

_, test_train_df = train_test_split(
    train_df,
    test_size=500,  # Number of items you want in your sample
    stratify=train_df['ac'],  # Stratify based on the combined column
    #random_state=42  # Ensures reproducibility
)

_, test_val_df = train_test_split(
    valid_df,
    test_size=100,  # Number of items you want in your sample
    stratify=valid_df['ac'],  # Stratify based on the combined column
    #random_state=42  # Ensures reproducibility
)

# create sub dataset
ds.create_sub_dataset(dataset_dir, test_train_df, test_val_df, class_names)

# augment dataset
ds.augment_dataset(dataset_dir, augmentation_metadata)

Removing dataset if pre-existing
Copying training files:


Copying files:   0%|          | 0/1000 [00:00<?, ?it/s]

Copying validation files:


Copying files:   0%|          | 0/200 [00:00<?, ?it/s]

Dataset 'test500' created at C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\YOLOv8\test500


Removing existing directories/files:   0%|          | 0/3 [00:00<?, ?it/s]

Copying files:   0%|          | 0/500 [00:00<?, ?it/s]

Copying files:   0%|          | 0/500 [00:00<?, ?it/s]

Applying augmentations:   0%|          | 0/250 [00:00<?, ?it/s]

An error occurred: Expected y_max for bbox (0.23570149999999995, 0.677776, 0.8784784999999999, 1.106294, 0) to be in the range [0.0, 1.0], got 1.106294.


# Training

In [9]:
# train on pure dataset

# dataset location
dataset_path=f'{dataset_dir}\\{dataset_name}.yaml'
project =  project_dir + 'pure' #save_dir # weight save path

# Create ClearML task
task = Task.init(
    project_name=project_name,
    task_name=f"{dataset_name}-pure"
)
task.set_parameter("model_variant", model_variant)

# Define Yolo model
#model = YOLO(f'{model_variant}.yaml') # train on model which is not pre-trained
model = YOLO(f'{model_variant}.pt')

#train args
args = dict(data=dataset_path, 
            epochs=epochs, 
            device=0, 
            close_mosaic=epochs, # disable mosaic augmentation
            seed=42
            )
task.connect(args)

# train model
results=model.train(**args, project=project)

# validate model
metrics = model.val()

# close task for next run
task.close()

#clear variables from memory
task = None
model = None
args = None
results = None


ClearML Task: created new task id=713de9427f1542e0887c9ad91a361338
2024-04-20 02:27:07,610 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/0e6b6f2d2ca946d584a5db9d951794d1/experiments/713de9427f1542e0887c9ad91a361338/output/log
2024-04-20 02:27:12,471 - clearml.model - INFO - Selected model id: 6932d54ba60d49f7bd3432493468f8c1
Ultralytics YOLOv8.2.2  Python-3.11.5 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16379MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=C:/github/Third-Year-Project/Intruder-Aircraft-Detection/datasets/YOLOv8/test500\test500.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=C:/github/Third-Year-Project/Intruder-Aircraft-Detection/logs/YOLOv8/test500/pure, name=train12, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=42, deterministic=True, single_cls=False, rect=Fa

train: Scanning C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\YOLOv8\test500\labels\train... 500 images, 0 backgrounds, 1 corrupt: 100%|██████████| 500/500 [00:00<00:00, 1922.89it/s]

train: WARNING  C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\YOLOv8\test500\images\train\58789.jpg: ignoring corrupt image/label: Label class 2 exceeds dataset class count 1. Possible class labels are 0-0
train: New cache created: C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\YOLOv8\test500\labels\train.cache


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\YOLOv8\test500\labels\valid... 100 images, 0 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<00:00, 586.66it/s]

...


20 epochs completed in 0.039 hours.
Optimizer stripped from C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\YOLOv8\test500\pure\train12\weights\last.pt, 6.2MB
Optimizer stripped from C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\YOLOv8\test500\pure\train12\weights\best.pt, 6.2MB

Validating C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\YOLOv8\test500\pure\train12\weights\best.pt...
Ultralytics YOLOv8.2.2  Python-3.11.5 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16379MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  4.99it/s]


                   all        100        100      0.683      0.776      0.697      0.334
Speed: 0.2ms preprocess, 1.7ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\YOLOv8\test500\pure\train12
2024-04-20 02:30:37,091 - clearml.storage - INFO - Starting upload: C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\YOLOv8\test500\pure\train12\weights\best.pt => https://files.clear.ml/Intruder-Aircraft-Detection/YOLOv8/test500-pure.713de9427f1542e0887c9ad91a361338/models/best.pt
Ultralytics YOLOv8.2.2  Python-3.11.5 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16379MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\YOLOv8\test500\labels\valid.cache... 100 images, 0 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:05<00:00,  1.18it/s]


                   all        100        100       0.69       0.78      0.711       0.34
Speed: 0.4ms preprocess, 6.2ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\YOLOv8\test500\pure\train122


In [10]:
# train on augmented dataset

# clear variables
torch.cuda.empty_cache()
gc.collect()

# dataset location
dataset_path=f'{dataset_dir}\\{dataset_name}-aug.yaml'
project =  project_dir + 'augmented' #save_dir # weight save path

# Create ClearML task
task = Task.init(
    project_name=project_name,
    task_name=f"{dataset_name}-augmented"
)
task.set_parameter("model_variant", model_variant) 

# Define Yolo model
#model = YOLO('yolov8n.yaml')
model = YOLO('yolov8n.pt')

# train model
results = model.train(data=dataset_path, 
                      epochs=epochs, 
                      device=0, 
                      project=project,
                      close_mosaic=epochs, # disable mosaic augmentation
                      seed=42)

# validate model
metrics = model.val()

# close task for next/last run
task.close()



ClearML Task: created new task id=abb9a272fad94dd1a6b63ec3dece9fb6
ClearML results page: https://app.clear.ml/projects/0e6b6f2d2ca946d584a5db9d951794d1/experiments/abb9a272fad94dd1a6b63ec3dece9fb6/output/log


Connecting multiple input models with the same name: `yolov8n`. This might result in the wrong model being used when executing remotely


Ultralytics YOLOv8.2.2  Python-3.11.5 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16379MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=C:/github/Third-Year-Project/Intruder-Aircraft-Detection/datasets/YOLOv8/test500\test500-aug.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=C:/github/Third-Year-Project/Intruder-Aircraft-Detection/logs/YOLOv8/test500/augmented, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=20, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False,

train: Scanning C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\YOLOv8\test500\labels\train-aug... 749 images, 0 backgrounds, 2 corrupt: 100%|██████████| 749/749 [00:00<00:00, 1050.27it/s]

train: WARNING  C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\YOLOv8\test500\images\train-aug\58789-flip.jpg: ignoring corrupt image/label: Label class 2 exceeds dataset class count 1. Possible class labels are 0-0
train: WARNING  C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\YOLOv8\test500\images\train-aug\58789.jpg: ignoring corrupt image/label: Label class 2 exceeds dataset class count 1. Possible class labels are 0-0
train: New cache created: C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\YOLOv8\test500\labels\train-aug.cache


...


20 epochs completed in 0.062 hours.
Optimizer stripped from C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\YOLOv8\test500\augmented\train5\weights\last.pt, 6.2MB
Optimizer stripped from C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\YOLOv8\test500\augmented\train5\weights\best.pt, 6.2MB

Validating C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\YOLOv8\test500\augmented\train5\weights\best.pt...
Ultralytics YOLOv8.2.2  Python-3.11.5 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16379MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  5.48it/s]


                   all        100        100      0.744      0.755      0.768      0.379
Speed: 0.2ms preprocess, 1.0ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\YOLOv8\test500\augmented\train5
Ultralytics YOLOv8.2.2  Python-3.11.5 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16379MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\YOLOv8\test500\labels\valid.cache... 100 images, 0 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:06<00:00,  1.13it/s]


                   all        100        100      0.758       0.76      0.774      0.377
Speed: 0.4ms preprocess, 3.2ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\YOLOv8\test500\augmented\train52
